<a href="https://colab.research.google.com/github/kplr-training/Airflow/blob/main/Ateliers/Solutions/12-Deferrable%20Operators.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deferrable operators :  Création de DAGs planifier en utilisant Async Operator 

![deff](https://user-images.githubusercontent.com/123757632/232047531-5cb354b1-ce74-4a95-af34-009febf1fe20.png)

Dag 1 : 

In [ ]:
from pendulum import datetime
from airflow import DAG
from airflow.sensors.date_time import DateTimeSensor

with DAG(
    "sync_dag_1",
    start_date=datetime(2023, 4, 8, 15, 30),
    end_date=datetime(2023, 4, 8, 15, 40),
    schedule="* * * * *",
    catchup=True,
) as dag:

Definir un DAG (Directed Acyclic Graph) Airflow nommé "sync_dag_1". Le DAG est planifié pour s'exécuter toutes les minutes, avec un horaire défini comme "* * * * *", ce qui signifie qu'il s'exécutera à chaque minute.

Le DAG est configuré pour commencer à s'exécuter le 8 avril 2023 à 15h30 et se terminer le même jour à 15h40. Le paramètre "catchup" est défini sur "True", ce qui signifie qu'Airflow exécutera les tâches précédentes manquées (si elles existent) lors du démarrage du DAG.

In [ ]:
sync_sensor = DateTimeSensor(
        task_id="sync_task",
        target_time="""{{ macros.datetime.utcnow() + macros.timedelta(minutes=10) }}""",
    )

Crée une instance de la classe DateTimeSensor d'Airflow appelée "sync_sensor", qui est utilisée pour attendre que la cible de temps spécifiée soit atteinte avant de passer à la tâche suivante dans le DAG.

Le paramètre "task_id" est défini sur "sync_task", ce qui est un identifiant unique pour la tâche.

Le paramètre "target_time" est défini en utilisant la syntaxe Jinja, qui est une syntaxe de modèle utilisée par Airflow pour définir des valeurs dynamiques. Dans ce cas, la cible de temps est définie comme "macros.datetime.utcnow() + macros.timedelta(minutes=10)", qui correspond à l'heure actuelle plus 10 minutes. Cela signifie que la tâche attendra que 10 minutes se soient écoulées depuis le début de l'exécution du DAG avant de passer à la tâche suivante.

En résumé, ce code crée une tâche de capteur de date et d'heure synchronisée qui attend que 10 minutes se soient écoulées avant de passer à la tâche suivante dans le DAG.





Execution du Dag : 

Sur la capture d'écran suivante, l'exécution du DAG produit 16 instances de tâches en cours d'exécution, chacune contenant un capteur de date et d'heure actif (DateTimeSensor) occupant un emplacement de travail (worker slot).

![image](https://user-images.githubusercontent.com/123757632/232053526-622ec6e5-01e8-4203-8dfd-3a72b02853a9.png)


Dag 2 : 

In [ ]:
from pendulum import datetime
from airflow import DAG
from airflow.sensors.date_time import DateTimeSensorAsync

with DAG(
    "async_dag_2",
    start_date=datetime(2023, 4, 8, 15, 30),
    end_date=datetime(2023, 4, 8, 15, 40),
    schedule="* * * * *",
    catchup=True,
) as dag:

Définir un DAG Airflow nommé "async_dag_2", qui est planifié pour s'exécuter toutes les minutes à l'aide de l'horaire "* * * * *". Le DAG commencera à s'exécuter le 8 avril 2023 à 15h30 et se terminera le même jour à 15h40.

Le paramètre "catchup" est défini sur "True", ce qui signifie qu'Airflow exécutera les tâches précédentes manquées (si elles existent) lors du démarrage du DAG.

In [ ]:
async_sensor = DateTimeSensorAsync(
        task_id="async_task",
        target_time="""{{ macros.datetime.utcnow() + macros.timedelta(minutes=10) }}""",
    )

Crée une instance de la classe DateTimeSensorAsync, qui est un capteur de temps asynchrone dans la bibliothèque Airflow. Le capteur attend que le temps cible soit atteint avant de déclencher une tâche.

L'instance est créée avec les arguments suivants :

- task_id : une chaîne qui identifie la tâche dans le DAG.
- target_time : une chaîne de modèles qui utilise les macros de date et d'heure d'Airflow pour définir le temps cible de ce capteur. Ici, le temps cible est défini comme "macros.datetime.utcnow() + macros.timedelta(minutes=10)", ce qui signifie que le capteur attendra 10 minutes à partir de l'heure actuelle avant de déclencher la tâche.

Une fois la tâche déclenchée, le flux de travail pourra continuer à exécuter les autres tâches qui dépendent de cette tâche.

Execution du Dag :


Sur la capture d'écran suivante, toutes les tâches sont affichées dans un état différé (violet). Les tâches dans d'autres DAG peuvent utiliser les emplacements de travail disponibles, ce qui rend l'opérateur différable plus rentable et plus efficace en termes de coûts et de temps.


![image](https://user-images.githubusercontent.com/123757632/232053153-47704f1c-77da-4bbf-850b-22b00fd34a52.png)